In [3]:
import pandas as pd
from pprint import pprint


data = (
    pd
    .read_csv('data/top_rated_wines.csv')
    .query('variety.notna()')
    .reset_index(drop=True)
    .to_dict('records')
)
pprint(data[:2])

[{'name': '3 Rings Reserve Shiraz 2004',
  'notes': 'Vintage Comments : Classic Barossa vintage conditions. An average '
           'wet Spring followed by extreme heat in early February. Occasional '
           'rainfall events kept the vines in good balance up to harvest in '
           'late March 2004. Very good quality coupled with good average '
           'yields. More than 30 months in wood followed by six months tank '
           'maturation of the blend prior to bottling, July 2007. ',
  'rating': 96.0,
  'region': 'Barossa Valley, Barossa, South Australia, Australia',
  'variety': 'Red Wine'},
 {'name': 'Abreu Vineyards Cappella 2007',
  'notes': 'Cappella is a proprietary blend of two clones of Cabernet '
           'Sauvignon with Cabernet Franc, Petit Verdot and Merlot. The '
           'gravelly soil at Cappella produces fruit that is very elegant in '
           'structure. The resulting wine exhibits beautiful purity of fruit '
           'with fine grained and lengthy

In [ ]:
! pip install qdrant-client
! pip install sentence-transformers
! pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 1.6 MB/s  0:00:01 eta 0:00:01


In [7]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

# create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance


# Create the embedding encoder
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings

In [8]:
# Create collection to store the wine rating data
collection_name="top_wines"

qdrant.recreate_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

/tmp/ipykernel_67888/2002094055.py:4: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [9]:
points_to_upload = []
for idx, doc in enumerate(data):
    points_to_upload.append(
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["notes"]).tolist(),
            payload=doc
        )
    )

# vectorize!
qdrant.upload_points(
    collection_name=collection_name,
    points=points_to_upload
)

In [10]:
user_prompt = "Suggest me an amazing Malbec wine from Argentina"

In [11]:
query_vector = encoder.encode(user_prompt).tolist()

In [12]:
# Search time for awesome wines!
from qdrant_client import QdrantClient
from qdrant_client.models import SearchParams, ScoredPoint

hits = qdrant.query_points(
    collection_name=collection_name,
    query=query_vector,
    limit=3
)

In [13]:
for hit in hits.points: # Corrected: iterate over hits.points to get the ScoredPoint objects
  pprint(hit)

ScoredPoint(id=293, version=0, score=0.6377782400762628, payload={'name': 'Catena Zapata Argentino Vineyard Malbec 2004', 'region': 'Argentina', 'variety': 'Red Wine', 'rating': 98.0, 'notes': '"The single-vineyard 2004 Malbec Argentino Vineyard spent 17 months in new French oak. Remarkably fragrant and complex aromatically, it offers up aromas of wood smoke, creosote, pepper, clove, black cherry, and blackberry. Made in a similar, elegant style, it is the most structured of the three single vineyard wines, needing a minimum of a decade of additional cellaring. It should easily prove to be a 25-40 year wine. It is an exceptional achievement in Malbec. When all is said and done, Catena Zapata is the Argentina winery of reference – the standard of excellence for comparing all others. The brilliant, forward-thinking Nicolas Catena remains in charge, with his daughter, Laura, playing an increasingly large role. The Catena Zapata winery is an essential destination for fans of both architect

In [20]:
# For Hugging Face models
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from huggingface_hub import login


# Log in to Hugging Face Hub (requires a token set in Colab secrets as 'HF_TOKEN')
# You can get a token from https://huggingface.co/settings/tokens and add it to Colab secrets.
try:
    hf_token =""
    if hf_token:
        login(token=hf_token)
        print("[green]Successfully logged into Hugging Face Hub.")
    else:
        print("Warning: Hugging Face token not found in Colab secrets. Some models might require authentication")
except Exception as e:
    print(f"Error during Hugging Face login: {e}. Some models might not load.")


# Set up device (GPU if available, else CPU)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load Llama 2 model and tokenizer
# Note: 'meta-llama/Llama-2-7b-chat-hf' is a large model (~13GB) and might require GPU or significant RAM.
# If you face memory issues, consider a smaller, similar model like 'TinyLlama/TinyLlama-1.1B-Chat-v1.0'
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0" # Changed to an openly accessible model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

prompt = [
    {"role": "system", "content": "You are a chatbot, a wine specialist. Your top priority is to help guide users into selecting amazing wine and guide them with their requests"},
    {"role": "user","content": user_prompt},
]
inputs = tokenizer.apply_chat_template(
	prompt,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=250)
pprint(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))




[green]Successfully logged into Hugging Face Hub.
Using device: cpu
("Certainly! Here's a suggestion for an amazing Malbec wine from Argentina:\n"
 '\n'
 '- Malbec from San Juan, Mendoza: This wine is made from the Malbec grape and '
 'is produced in the San Juan region of Mendoza. It has a deep purple color, '
 'notes of black cherry, blackberry, and plum, with a rich, full-bodied flavor '
 "and a long, smooth finish. It's a great choice for those who love bold, "
 'full-bodied wines with complex flavors.\n'
 '\n'
 'This wine is available in many stores and online wine shops in Argentina and '
 'around the world. You can also try it at a local wine bar or restaurant. '
 'Enjoy!</s>')


In [22]:
# define a variable to hold the search results
search_results = [hit.payload for hit in hits.points]

In [24]:
# For Hugging Face models
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from huggingface_hub import login



# Load Llama 2 model and tokenizer
# Note: 'meta-llama/Llama-2-7b-chat-hf' is a large model (~13GB) and might require GPU or significant RAM.
# If you face memory issues, consider a smaller, similar model like 'TinyLlama/TinyLlama-1.1B-Chat-v1.0'
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0" # Changed to an openly accessible model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

prompt = [
    {"role": "system", "content": "You are a chatbot, a wine specialist. Your top priority is to help guide users into selecting amazing wine and guide them with their requests"},
    {"role": "user","content": user_prompt},
    {"role": "assistant", "content": str(search_results)},
]
inputs = tokenizer.apply_chat_template(
	prompt,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=250)
pprint(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

('Based on the given text, I suggest the following Malbec wine from '
 'Argentina:\n'
 '\n'
 "[{'name': 'Catena Zapata Adrianna Vineyard Malbec 2004', 'region': "
 "'Argentina', 'variety': 'Red Wine', 'rating': 97.0, 'notes': "
 '\'"The single-vineyard 2004 Malbec Adrianna Vineyard from the Gualtallary '
 'district is inky purple with aromas of wood smoke, pencil lead, game, black '
 'cherry, and blackberry liqueur. Opulent, full-flavored, yet remarkably light '
 'on its feet, this medium to full-bodied Malbec is all about pleasure. It '
 'will certainly evolve for a decade but is hard to resist now. It is a fine '
 "test of one's ability to defer immediate gratification. When all is said and "
 'done, Catena Zapata is the Argentina winery of reference – the standard of '
 'excellence for comparing all others. The brilliant, forward-thinking Nicolas '
 'Catena remains in charge, with his daughter, Laura, playing an increasingly '
 'large')
